In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

In [2]:
loader = CSVLoader('data_input/test.csv')
big_df = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True
)

In [4]:
chunks = text_splitter.split_documents(big_df)

In [5]:
len(chunks)

73

In [6]:
embedings = OllamaEmbeddings(
    model='all-minilm'
)

In [7]:
from exp import create_folder_for_db
name = 'vector'
create_folder_for_db(name)

chromadb = Chroma.from_documents(chunks, embedings, persist_directory = name, collection_name = name)

In [21]:
# Prompt
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Conversation_history: {conversation_history}

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [23]:
# Chain
MODEL = 'llama2'
model = Ollama(model=MODEL)
parser = StrOutputParser()
retriever = chromadb.as_retriever()

from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "conversation_history": itemgetter("conversation_history"),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [53]:
query = 'what happend in  August 27  1991'
history = ''

answer = chain.invoke({'question': query, "conversation_history": history})
print(answer)

Based on the information provided in the CSV file, on August 27, 1991, Moldova declared its independence from the Soviet Union.


In [50]:
query = 'when Moldova becomes independent from the Soviet Union?'
source = chromadb.similarity_search(query)
model.invoke(f"based in this info: {source}, tell me {query}")

'Based on the provided information, Moldova became independent from the Soviet Union on August 27, 1991.'